In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics
from matplotlib import pyplot as plt

df = pd.read_csv("bikebuyers_original.csv")
df

,ID,Marital Status,Gender,Income,Children,Cars,Age,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
0,19147,Married,Male,40000,0,1,42,Bachelors,Professional,No,0-1 Miles,North America,No
1,22296,Married,Male,70000,0,1,38,Bachelors,Professional,No,0-1 Miles,North America,No
2,25983,Married,Male,70000,0,1,43,Bachelors,Professional,No,0-1 Miles,North America,No
3,29134,Married,Male,60000,4,3,42,Bachelors,Skilled Manual,No,10+ Miles,North America,No
4,27388,Married,Male,60000,3,2,66,Bachelors,Management,No,1-2 Miles,North America,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,14798,Single,Female,10000,4,2,41,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
996,16259,Single,Female,10000,4,2,40,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
997,16410,Single,Female,10000,4,2,41,Partial High School,Manual,Yes,0-1 Miles,Europe,Yes
998,28957,Single,Female,120000,0,4,34,Partial High School,Professional,Yes,10+ Miles,Pacific,Yes
